In [1]:
import os
from crewai import Agent, Task, Crew, Process
import pandas as pd

In [2]:
df = pd.read_csv("../emotions_train.csv")
# remove label column
df = df.drop("label", axis=1)
# convert df into list of tuples of (text, label) pairs
data = df.to_records(index=False).tolist()
# convert it to a list of strings
data = [str(i[0]) + ":" + str(i[1]) for i in data]

In [3]:
df

,text,label_name
0,i stated in the class discussions the school d...,sadness
1,i feel that he is ungrateful for having an opp...,sadness
2,i feel honored and humbled by this turn of events,joy
3,i feel so damn complacent,joy
4,i still love to run and plan to keep it up but...,love
5,i start to hate the fact that whenever i post ...,love
6,im feeling really annoyed,anger
7,i really shut myself off a lot at the end of m...,anger
8,i feel so uncertain about the decade ahead,fear
9,i feel like i m uncertain about things i was o...,fear


In [4]:
data

['i stated in the class discussions the school discourages the use of im reference at the library because they feel that it will be abused:sadness',
 'i feel that he is ungrateful for having an opportunity to breathe the air when so many others didn t have the chances he has had:sadness',
 'i feel honored and humbled by this turn of events:joy',
 'i feel so damn complacent:joy',
 'i still love to run and plan to keep it up but i don t want to once again register for so many races that i feel like every exercise moment needs to be devoted to running:love',
 'i start to hate the fact that whenever i post anything it would eventually end up with me writing about how lonely i feel because i have no romantic partner whatsoever:love',
 'im feeling really annoyed:anger',
 'i really shut myself off a lot at the end of my pregnancy when i was feeling even more irritable and anxious so maybe now i need to bounce back from that and get back to normal again:anger',
 'i feel so uncertain about the 

In [5]:
os.environ["OPENAI_API_BASE"] = "https://api.fireworks.ai/inference/v1"
os.environ["OPENAI_MODEL_NAME"] = (
    "accounts/fireworks/models/llama-v3-70b-instruct"  # Adjust based on available model
)
os.environ["OPENAI_API_KEY"] = "GOtyI1w6nujfkksUvHvCiFHnDaAVP2HkTiDIxo4jwXALLZLo"

# Agetns

In [12]:
creator = Agent(
    role="Dataset Creator",
    goal=f"You're given some rows of a emotion classification dataset, your goal is to write new rows for the dataset. each row is (text, label_name)",
    backstory="""You're working as data scientist at a company that is building a model to classify emotions. You're given a dataset with some rows and you need to write new rows for the dataset.""",
    verbose=True,
    allow_delegation=False,
)

In [13]:
labeler = Agent(
    role="Check if the labels are correct",
    goal="Label text data with the correct emotion label from the list of emotions: 'sadness', 'joy', 'anger', 'fear', 'surprise', 'love'.",
    backstory=""" You're a very good data annotator and you're given a dataset with some rows and you need to label the text data with the correct emotion label from the list of emotions: 'sadness', 'joy', 'anger', 'fear', 'surprise', 'love'.""",
    verbose=True,
    allow_delegation=True,
)

# Tasks

In [14]:
task1 = Task(
    description=f"""Given this sample data: {data} Generate new 10 rows similar to this data""",
    expected_output="10 rows of data (text, emotion label)",
    agent=creator,
)

In [15]:
task2 = Task(
    description="""Given series of data: (text, emotion label) Check if the labels are correct""",
    expected_output="list of correct labels for each text data",
    agent=labeler,
    context=[task1],
)

# Crew

In [16]:
crew = Crew(
    agents=[creator, labeler],
    tasks=[task1, task2],
    verbose=2,
)

2024-05-04 21:12:52,569 - 140182395585600 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [18]:
df

,text,label_name
0,i stated in the class discussions the school d...,sadness
1,i feel that he is ungrateful for having an opp...,sadness
2,i feel honored and humbled by this turn of events,joy
3,i feel so damn complacent,joy
4,i still love to run and plan to keep it up but...,love
5,i start to hate the fact that whenever i post ...,love
6,im feeling really annoyed,anger
7,i really shut myself off a lot at the end of m...,anger
8,i feel so uncertain about the decade ahead,fear
9,i feel like i m uncertain about things i was o...,fear


'i feel frustrated when people don't listen to me:anger', 

'i'm so grateful for the opportunity to travel and experience new cultures:joy', 

'i'm still trying to process the news and i'm feeling overwhelmed:sadness', 

'i'm really excited to start this new chapter in my life:joy', 

'i'm scared of what the future holds for me and my family:fear', 

'i'm so disappointed in myself for not speaking up sooner:sadness', 

'i'm feeling a sense of relief now that the project is finally complete:surprise', 

'i'm so proud of myself for pushing through the difficult times:love', 

'i'm feeling really anxious about the upcoming exam: fear', 

'i'm so happy to be surrounded by loving friends and family:joy'<|eot_id|><|start_header_id|>assistant<|end_header_id|>

In [17]:
result = crew.kickoff()

 [DEBUG]: == Working Agent: Dataset Creator
 [INFO]: == Starting Task: Given this sample data: ['i stated in the class discussions the school discourages the use of im reference at the library because they feel that it will be abused:sadness', 'i feel that he is ungrateful for having an opportunity to breathe the air when so many others didn t have the chances he has had:sadness', 'i feel honored and humbled by this turn of events:joy', 'i feel so damn complacent:joy', 'i still love to run and plan to keep it up but i don t want to once again register for so many races that i feel like every exercise moment needs to be devoted to running:love', 'i start to hate the fact that whenever i post anything it would eventually end up with me writing about how lonely i feel because i have no romantic partner whatsoever:love', 'im feeling really annoyed:anger', 'i really shut myself off a lot at the end of my pregnancy when i was feeling even more irritable and anxious so maybe now i need to bou

KeyboardInterrupt: 